In [29]:
import os
import torch
import numpy as np
import sys
sys.path.append(os.path.join(os.path.dirname('__file__'), '../'))
from Utils.context_fid import Context_FID
from Utils.metric_utils import display_scores
from Utils.cross_correlation import CrossCorrelLoss

In [30]:
ori_data = np.load('../Data/samples_24/mujoco_norm_truth.npy')
fake_data = np.load('../Data/ldm_fake_mujoco_14_2_32_3.npy')
iterations = 5

In [31]:
## Context-FID Score
context_fid_score = []
for i in range(iterations):
    context_fid = Context_FID(ori_data[:], fake_data[:])
    context_fid_score.append(context_fid)
    print(f'Iter {i}: ', 'context-fid =', context_fid, '\n')
      
display_scores(context_fid_score)

Iter 0:  context-fid = 0.011148500333768324 

Iter 1:  context-fid = 0.010691388026714672 

Iter 2:  context-fid = 0.012643917935533919 

Iter 3:  context-fid = 0.012426008928757563 

Iter 4:  context-fid = 0.011066832231001886 

Final Score:  0.01159532949115527 ± 0.0010905531918158462


In [32]:
## Correlational Score
def random_choice(size, num_select=100):
    select_idx = np.random.randint(low=0, high=size, size=(num_select,))
    return select_idx

x_real = torch.from_numpy(ori_data)
x_fake = torch.from_numpy(fake_data)

correlational_score = []
size = int(x_real.shape[0] / iterations)

for i in range(iterations):
    real_idx = random_choice(x_real.shape[0], size)
    fake_idx = random_choice(x_fake.shape[0], size)
    corr = CrossCorrelLoss(x_real[real_idx, :, :], name='CrossCorrelLoss')
    loss = corr.compute(x_fake[fake_idx, :, :])
    correlational_score.append(loss.item())
    print(f'Iter {i}: ', 'cross-correlation =', loss.item(), '\n')

display_scores(correlational_score)

Iter 0:  cross-correlation = 0.17322828910185467 

Iter 1:  cross-correlation = 0.1841562181970415 

Iter 2:  cross-correlation = 0.16532368575974365 

Iter 3:  cross-correlation = 0.21915195786787223 

Iter 4:  cross-correlation = 0.1624249375772327 

Final Score:  0.18085701770074897 ± 0.02856239240159226


In [33]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import sys
sys.path.append(os.path.join(os.path.dirname('__file__'), '../'))

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

from Utils.metric_utils import display_scores
from Utils.discriminative_metric import discriminative_score_metrics
from Utils.predictive_metric import predictive_score_metrics

In [34]:
### Discriminative score
discriminative_score = []

for i in range(iterations):
    temp_disc, fake_acc, real_acc = discriminative_score_metrics(ori_data[:], fake_data[:])
    discriminative_score.append(temp_disc)
    print(f'Iter {i}: ', temp_disc, ',', fake_acc, ',', real_acc, '\n')
      
print('sine:')
display_scores(discriminative_score)
print()

training:   0%|          | 0/2000 [00:00<?, ?it/s]

Iter 0:  0.010499999999999954 , 0.528 , 0.493 



training:   0%|          | 0/2000 [00:00<?, ?it/s]

Iter 1:  0.006000000000000005 , 0.496 , 0.492 



training:   0%|          | 0/2000 [00:00<?, ?it/s]

Iter 2:  0.0030000000000000027 , 0.485 , 0.509 



training:   0%|          | 0/2000 [00:00<?, ?it/s]

Iter 3:  0.0040000000000000036 , 0.5175 , 0.4905 



training:   0%|          | 0/2000 [00:00<?, ?it/s]

Iter 4:  0.003500000000000003 , 0.552 , 0.441 

sine:
Final Score:  0.005399999999999993 ± 0.0038119286969725843



In [35]:
### Predictive score
predictive_score = []
for i in range(iterations):
    temp_pred = predictive_score_metrics(ori_data, fake_data[:])
    predictive_score.append(temp_pred)
    print(i, ' epoch: ', temp_pred, '\n')
      
print('sine:')
display_scores(predictive_score)
print()

training:   0%|          | 0/5000 [00:00<?, ?it/s]

0  epoch:  0.008206780216246892 



training:   0%|          | 0/5000 [00:00<?, ?it/s]

1  epoch:  0.008861996643815576 



training:   0%|          | 0/5000 [00:00<?, ?it/s]

2  epoch:  0.009296916069707156 



training:   0%|          | 0/5000 [00:00<?, ?it/s]

3  epoch:  0.007417622583749451 



training:   0%|          | 0/5000 [00:00<?, ?it/s]

4  epoch:  0.008274076756341033 

sine:
Final Score:  0.008411478453972021 ± 0.0008857461493286903

